<a href="https://colab.research.google.com/github/AlexanderMaslikhin/parsing/blob/homework4/parsing_homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.parse as up
from pprint import pprint
import re
import json

In [3]:
def get_vacancies_hh(vacancy, pages_count=1):
  vlist = []
  headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
  }
  for p in range(pages_count):
    if not p:
        url = f"https://hh.ru/search/vacancy?text={up.quote_plus(vacancy, safe='')}&from=suggest_post&area=1"
    else:
        url = f"https://hh.ru/search/vacancy?text={up.quote_plus(vacancy, safe='')}&from=suggest_post&area=1&page={p}&hhtmFrom=vacancy_search_list"
    response = requests.get(url, headers=headers)
    dom = bs(response.text, "html.parser")
    v_list_on_page = dom.select("div.vacancy-serp-item-body")

    for vacansy_block in v_list_on_page:
        v_dict = {}
        a_header = vacansy_block.select("a.serp-item__title")
        v_dict['name'] = a_header[0].text
        v_dict['link'], _ = a_header[0]["href"].split('?')
        salary_block = vacansy_block.select("span.bloko-header-section-3")

        if salary_block:
          salary = {}
          salary_text = re.sub(r'\s', '', salary_block[0].text)
          if salary_text.startswith('от'):
            m = re.search(r'\d+', salary_text)
            salary['from'] = int(m.group(0))
            salary['currency'] = re.sub(r'\d+', '', salary_text[2:])
          elif salary_text.startswith('до'):
            m = re.search(r'\d+', salary_text)
            salary['to'] = int(m.group(0))
            salary['currency'] = re.sub(r'\d+', '', salary_text[2:])
          else:
            salary['from'], salary['to'] = map(int, re.split(r'\D+', salary_text)[:-1])
            salary['currency'] = re.sub(r'\d+–\d+', '', salary_text)
          v_dict['salary'] = salary
        v_dict['employer'] = vacansy_block.select('a.bloko-link.bloko-link_kind-tertiary')[0].text
        v_dict['place'] = vacansy_block.find('div', attrs={"data-qa": "vacancy-serp__vacancy-address"}).text

        vlist.append(v_dict)

  return vlist    

vacancies = get_vacancies_hh("Python разработчик", pages_count=5)


1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [6]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

In [7]:
def db_insert_new_vacancies(v_list, collection):
  vacancies_added = 0
  for v in v_list:
    v["_id"] = v["link"].split("/")[-1]
    try:
      collection.insert_one(v)
      vacancies_added += 1
    except DuplicateKeyError:
      pass
  return vacancies_added

  

In [19]:
client = MongoClient("37.140.199.115:27017")
db = client["vacancies"]
collection = db["hh"]
added = db_insert_new_vacancies(vacancies, collection)
print(f'{added} new vacancies was inserted')

94 new vacancies was inserted


In [20]:
# добавим еще 2 страницы в парсинг 
new_vacancies = get_vacancies_hh("Python разработчик", pages_count=7)
added = db_insert_new_vacancies(new_vacancies, collection)
print(f'{added} new vacancies was inserted')

75 new vacancies was inserted


In [21]:
collection.find_one()

{'_id': '77147428',
 'name': 'Backend программист (Python)',
 'link': 'https://hh.ru/vacancy/77147428',
 'salary': {'from': 120000, 'currency': 'руб.'},
 'employer': 'MANGO FZCO',
 'place': 'Москва'}

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

In [26]:
from pprint import pprint

def db_search_salary(more_then, collection):
  return collection.find({'$or': [{'salary.from': {'$gte': more_then}}, {'salary.to': {'$gte': more_then}}]})

vacancies_more_200 = db_search_salary(200000, collection)
for v in vacancies_more_200:
  pprint(v)

{'_id': '77091465',
 'employer': 'ООО\xa0Нетопия',
 'link': 'https://hh.ru/vacancy/77091465',
 'name': 'Разработчик Python',
 'place': 'Москва',
 'salary': {'currency': 'руб.', 'from': 200000, 'to': 250000}}
{'_id': '70705466',
 'employer': 'ООО\xa0Фабрика Решений',
 'link': 'https://hh.ru/vacancy/70705466',
 'name': 'Python-разработчик (Django, DRF)',
 'place': 'Москва',
 'salary': {'currency': 'руб.', 'from': 60000, 'to': 220000}}
{'_id': '72216524',
 'employer': 'ООО\xa0Перфект Системс',
 'link': 'https://hh.ru/vacancy/72216524',
 'name': 'Python Backend Developer / WEB - разработчик',
 'place': 'Москва',
 'salary': {'currency': 'руб.', 'from': 300000, 'to': 450000}}
{'_id': '71147348',
 'employer': 'ООО\xa0Дубайт',
 'link': 'https://hh.ru/vacancy/71147348',
 'name': 'Разработчик C# / Python',
 'place': 'Москва, Проспект Вернадского и еще\xa02\xa0',
 'salary': {'currency': 'руб.', 'from': 300000, 'to': 500000}}
{'_id': '77179552',
 'employer': 'ООО\xa024 часа ТВ',
 'link': 'https://